# logging.torch

> 检查PyTorch模型是否符合预期，是否为要训练的模型

In [ ]:
#| default_exp logging.torch

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from scholarly_infrastructure.logging.nucleus import logger

In [ ]:
#| export
from fastcore.basics import patch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from rich.table import Table

@patch
def inspect_model_parameters(model:nn.Module):
    trainable_params = 0
    all_param = 0
    trainable_bytes = 0
    all_bytes = 0
    for _, param in model.named_parameters():
        param_bytes = param.numel() * param.element_size()
        all_param += param.numel()
        all_bytes += param_bytes
        if param.requires_grad:
            trainable_params += param.numel()
            trainable_bytes += param_bytes
    return trainable_params, all_param, trainable_bytes, all_bytes

@patch
def num_of_total_parameters(model:nn.Module):
    return (model).inspect_model_parameters()[1]

@patch
def num_of_trainable_parameters(model:nn.Module):
    return (model).inspect_model_parameters()[0]

@patch
def print_trainable_parameters(model:nn.Module):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params, all_param, trainable_bytes, all_bytes = model.inspect_model_parameters()
    # print(
    table = Table(title=f"Model {model.__class__.__name__}'s Trainable Parameters Inspection")
    table.add_column("Number of Trainable Parameters", justify="right", style="cyan", no_wrap=True)
    table.add_column("Number of Total Parameters", style="magenta")
    table.add_column("Trainable Ratio (0-1)", justify="right", style="green")
    table.add_row(f"{trainable_params:.3e} ({trainable_bytes:.3e} bytes)", f"{all_param:.3e} ({all_bytes:.3e} bytes)", f"{trainable_params / all_param:.3e}")
    
    logger.info(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}", 
        rich=table
    )

In [ ]:
from transformers import AutoModel, AutoConfig
test_model = AutoModel.from_config(AutoConfig.from_pretrained("google/vit-base-patch16-224-in21k"))

In [ ]:
test_model.print_trainable_parameters()

Sat 2024-11-16 16:46:31.180441

INFO                         Model ViTModel's Trainable Parameters Inspection                      ]8;id=688599;file:///tmp/ipykernel_2289193/1218786521.py\1218786521.py]8;;\:]8;id=421080;file:///tmp/ipykernel_2289193/1218786521.py#45\45]8;;\
         ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓                  
         ┃ Number of Trainable Parameters ┃ Number of Total Parameters  ┃ Trainable Ratio (0-1) ┃                  
         ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩                  
         │    8.639e+07 (3.456e+08 bytes) │ 8.639e+07 (3.456e+08 bytes) │             1.000e+00 │                  
         └────────────────────────────────┴─────────────────────────────┴───────────────────────┘                  

In [ ]:
#| export
from bigmodelvis import Visualization
@patch
def model_rich_tree(self:nn.Module):
    module_tree = Visualization(self).structure_graph(printTree=False)
    return module_tree

from rich.panel import Panel
@patch
def print_model_pretty(self:nn.Module):
    module_tree = self.model_rich_tree()
    panel = Panel(module_tree, title=f"Model Tree for {self.__class__.__name__}")
    logger.info(str(self), rich=panel)
    # return module_tree

In [ ]:
test_model.print_model_pretty()

Sat 2024-11-16 16:46:31.346081

INFO     ╭─────────────────────────────── Model Tree for ViTModel ────────────────────────────────╮ ]8;id=728585;file:///tmp/ipykernel_2289193/640056773.py\640056773.py]8;;\:]8;id=216835;file:///tmp/ipykernel_2289193/640056773.py#13\13]8;;\
         │ root                                                                                   │                
         │ ├── embeddings (ViTEmbeddings) cls_token:[1, 1, 768] position_embeddings:[1, 197, 768] │                
         │ │   └── patch_embeddings (ViTPatchEmbeddings)                                          │                
         │ │       └── projection (Conv2d) weight:[768, 3, 16, 16] bias:[768]                     │                
         │ ├── encoder (ViTEncoder)                                                               │                
         │ │   └── layer (ModuleList)                                                             │                
         │ │       └── 0-11(ViTLayer)                                                             │                
         │ │           ├── attention (ViTSdpaAttention)                                           │                
         │ │           │   ├── attention (ViTSdpaSelfAttention)                                   │                
         │ │           │   │   └── query,key,value(Linear) weight:[768, 768] bias:[768]           │                
         │ │           │   └── output (ViTSelfOutput)                                             │                
         │ │           │       └── dense (Linear) weight:[768, 768] bias:[768]                    │                
         │ │           ├── intermediate (ViTIntermediate)                                         │                
         │ │           │   └── dense (Linear) weight:[3072, 768] bias:[3072]                      │                
         │ │           ├── output (ViTOutput)                                                     │                
         │ │           │   └── dense (Linear) weight:[768, 3072] bias:[768]                       │                
         │ │           └── layernorm_before,layernorm_after(LayerNorm) weight:[768] bias:[768]    │                
         │ ├── layernorm (LayerNorm) weight:[768] bias:[768]                                      │                
         │ └── pooler (ViTPooler)                                                                 │                
         │     └── dense (Linear) weight:[768, 768] bias:[768]                                    │                
         ╰────────────────────────────────────────────────────────────────────────────────────────╯                

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()